In [123]:
import numpy as np
import numba as nb
import matplotlib.pyplot as plt
import time
from math import sin
from numpy import sum, zeros
import copy


# Helper Functions
def finite(N: int, c: float) -> np.ndarray:
    """
    Generates the adjacency matrix of the finite connectivity graph

    Args:
        N (int): Number of XY spins in the network
        c (float): Average connectivity

    Returns:
        array: Finite connectivity graph adjacency matrix
    """
    A = np.random.uniform(0, 1, size=(N, N))
    A_mask = np.triu((A < c/N).astype(int), k=1)
    A_symm = (A_mask + A_mask.T) 
    return A_symm


def ring(N: int) -> np.ndarray:
    """
    Generates the adjacency matrix of the ring

    Args:
        N (int): Number of XY spins in the network

    Returns:
        array: Ring adjacency matrix
    """
    A = np.zeros((N, N))
    pos = 0

    for i in range(N):
        A[i][(pos+1) % N] = 1
        A[i][(pos-1) % N] = 1
        pos += 1
    return A.astype(int)
    

# Network Parameters
N = 1000            # Numbers of Spings
c = 10              # Finite Connectivity
J = 0.0             # Finite Graph Strength
J0 = 1.0            # Ring Strength

# Simulation Parameters
T = 1               # Temperature
dt = 0.1           # Time Step
n = int(np.sqrt(N)) # Number of Runs (computed as sqrt(N) for proper stats)
n_s = int((N/dt))   # Number of Steps Per Run (computed to allow enough time for system to evolve)
b_n = 1000          # Burn-in Time

# Simulation Containers
s_0 = np.random.uniform(-1,1, N)*np.pi # Initial State
s_n = np.zeros((n, N))            # Samples
A_r = ring(N)
A_f = finite(N, c)


In [73]:
def force(spins: np.ndarray, A_r: np.ndarray, A_f: np.ndarray, J: float, J0: float) -> np.ndarray:
    """
    Computes the force vector for the current state of the system

    Returns:
        array: _description_
    """
    s_outer = np.subtract.outer(spins, spins)
    s_ring = s_outer * A_r
    s_finite = s_outer * A_f
    f = np.add(J0 * np.sum(np.sin(s_ring), axis=0), J * np.sum(np.sin(s_finite), axis=0))
    return f

In [148]:
@nb.njit(parallel=True, cache=True)
def force2(spins: np.ndarray, A_r: np.ndarray, A_f: np.ndarray, J: float, J0: float) -> np.ndarray:
    t = zeros((N,N))
    for i in range(N):
        for j in range(N):
            t[i, j] = spins[i] - spins[j]
    t = np.sin(t)
    return J*sum(t*A_f, axis=0) + J0*sum(t*A_r, axis=0)

In [150]:
t_0 = time.time()
a = force2(s_0, A_r, A_f, J, J0)
print(time.time()-t_0)

t_0 = time.time()
b = force(s_0, A_r, A_f, J, J0)
print(time.time()-t_0)


0.013799190521240234
0.02227497100830078


In [195]:
@nb.njit(parallel=True)
def loop(s: np.ndarray, n: int, A_r: np.ndarray, A_f: np.ndarray, J: float, J0: float, T:float, dt: float) -> np.ndarray:
    #np.random.seed()
    l = s.size
    for i in range(n):
        noise = np.empty(l)
        for ii in range(l):
            noise[ii] = np.random.normal()
        s += (force2(s, A_r, A_f, J, J0)*dt +  noise* np.sqrt(2 * T * dt))

    return s

In [198]:
t0 = time.time()
t = loop(s_0, 100, A_r, A_f, J, J0, T, dt)
print(time.time()-t0)

t0 = time.time()
t = loop2(s_0, 100, A_r, A_f, J, J0, T, dt)
print(time.time()-t0)

1.1574270725250244
1.0986077785491943


In [192]:
def loop2(s: np.ndarray, n: int, A_r: np.ndarray, A_f: np.ndarray, J: float, J0: float, T:float, dt: float) -> np.ndarray:
    for i in range(n):
        s += (force2(s, A_r, A_f, J, J0)*dt +  np.random.normal(size=s.size)* np.sqrt(2 * T * dt))

    return s

In [173]:
np.random.normal()

0.35489165840916476

In [175]:
s_0.size

1000